## Training Unpruned Model [Not Our Approach]

In [1]:
import torch
import torch.optim as optim
from get_loader import get_loader
from torchvision import transforms
import New_Pruned_Model
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from utils import print_examples
from New_Pruned_Model import EncoderCNN

In [2]:
mse_loss = nn.MSELoss()
def match_hidden_states(sub_network_hidden_states, decoder_hidden_states , num_selected_layers = 3):
    # Here we have to select the layers coming with decoder network
    num_total_layers = len(decoder_hidden_states)
    selected_layers_indices = [num_selected_layers - 1]
    for i in range(num_selected_layers - 1):
        selected_layers_indices.append(int((num_total_layers - 1) * (i + 1)/num_selected_layers))
    
    selected_decoder_hidden_states = [decoder_hidden_states[idx] for idx in selected_layers_indices]
    loss = 0 
    for sub_state, dec_state in zip(sub_network_hidden_states, selected_decoder_hidden_states):
        loss += mse_loss(sub_state, dec_state)
    return loss

In [3]:
def train():
    mse_loss_2 = nn.MSELoss()
    transform = transforms.Compose([transforms.Resize((350,350)),transforms.RandomCrop((256,256)),
                                transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    images_path , caption_path = r"D:\ML\Korea\Jishu\Jishu\rsicd\images" , r"D:\ML\Korea\Jishu\Jishu\rsicd\captions.csv"
    pruned_resnet_model_path = r"D:\ML\Korea\Jishu\Jishu\Cnn_Pruning\Pruned_Resnet\fine_tuned_model.pth"
    
    BATCH_SIZE = 32
    data_loader , dataset = get_loader(images_path,caption_path ,transform,batch_size = BATCH_SIZE,num_workers=4)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_epochs = 20
    learning_rate = 3e-4
    trg_vocab_size = len(dataset.vocab)
    embedding_size = 512
    num_heads = 8
    num_decoder_layers = 2
    dropout = 0.10
    pad_idx=dataset.vocab.stoi["<PAD>"]
    save_model = True
    writer =SummaryWriter("runs/loss_plot")
    step = 0
    encoder_regularization_penalty = 0.01
    decoder_regularizartion_penalty = 0.01 
    
    model = New_Pruned_Model.EncodertoDecoder(embeding_size=embedding_size, trg_vocab_size=trg_vocab_size, num_heads=num_heads, num_decoder_layers=4, dropout=dropout).to(device)
    model.load_state_dict(torch.load(r'D:\ML\Korea\Jishu\Jishu\Final_Docs\Complete_Model_Pruning\model.pth', map_location=device))
    model = model.to(device)
    
    # Now we define the pruned model
    pruned_model = New_Pruned_Model.PrunedEncodertoDecoder(embeding_size=embedding_size,
                            trg_vocab_size=trg_vocab_size, num_heads=num_heads,
                            num_decoder_layers=num_decoder_layers,
                            dropout=dropout , pruned_resnet_model_path = pruned_resnet_model_path).to(device)
    
    
    optimizer = optim.Adam(pruned_model.parameters(),lr = learning_rate)
    criterion2 = nn.CrossEntropyLoss(ignore_index=pad_idx)
    
    l = []
    for epoch in range(num_epochs):
        print(f"[Epoch {epoch} / {num_epochs}]")
        model.eval()
        pruned_model.train()
        Total_loss = 0.0
        for idx, (images, captions) in tqdm(enumerate(data_loader), total=len(data_loader), leave=False):
            images = images.to(device)
            captions = captions.to(device)
            with torch.no_grad():
                output , hidden_original_decoder_outputs = model(images, captions[:-1])
                
            pruned_model_outputs , hidden_pruned_decoder_outputs = pruned_model(images , captions[:-1])
            pruned_model_outputs = pruned_model_outputs.reshape(-1 , pruned_model_outputs.shape[2])
            target = output.reshape(-1 , output.shape[2]).to(device)
            
            optimizer.zero_grad()
            # print(pruned_model_outputs.shape , target.shape)
            loss_match = mse_loss_2(pruned_model_outputs , target)
            
            # Compute the L2 Regularization loss of the encoder weights
            l2_reg = pruned_model.encoder.compute_penalty(encoder_regularization_penalty)
            mse_loss = match_hidden_states(hidden_pruned_decoder_outputs,hidden_original_decoder_outputs)
            dec_loss = mse_loss * decoder_regularizartion_penalty
            
            # Now calculate the total loss
            total_loss = loss_match + l2_reg + dec_loss
            lossofepoch = total_loss.item()
            Total_loss += lossofepoch
            total_loss.backward()
            
            torch.nn.utils.clip_grad_norm_(pruned_model.parameters(),max_norm=1)
            optimizer.step()
            writer.add_scalar("Training Loss",Total_loss,global_step=step)
            step+=1
            
        LOSS = Total_loss / len(data_loader)
        l.append(LOSS) 
        print("Loss of the epoch is", Total_loss / len(data_loader))
        torch.save(pruned_model , 'unpruned_model_final.pth')
            

        pruned_model.eval()
        print_examples(pruned_model, device, dataset)

In [4]:
train()

c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is F

[Epoch 0 / 20]


  0%|          | 0/1278 [00:00<?, ?it/s]c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Loss of the epoch is 31.23588681407564


c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


...

c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Example 1 CORRECT:  <SOS> many buildings and green trees are in a dense residential area . <EOS> 


Example 1 OUTPUT:  many green trees and some buildings are in a resort . <EOS>
Example 1 CORRECT:  <SOS> it 's a big piece of naked dirt . <EOS> 


Example 1 OUTPUT:  the desert is very dry . <EOS>


[Epoch 19 / 20]


Loss of the epoch is 0.7317658195006829


c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Example 1 CORRECT:  <SOS> an island on a lake surrounded by many green trees are in a park . <EOS> 


Example 1 OUTPUT:  many green trees and some buildings are in a resort . <EOS>
Example 1 CORRECT:  <SOS> the water in the sea is very rough . <EOS> 


Example 1 OUTPUT:  a large number of trees are planted around the lake . <EOS>


